# Debugging scikit-learn text classification pipeline




## 1. Baseline model

First, we need some data. Let's load 20 Newsgroups data, keeping only 4 categories:

In [ ]:
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'soc.religion.christian', 
              'comp.graphics', 'sci.med']
twenty_train = fetch_20newsgroups(
    subset='train',
    categories=categories,
    shuffle=True,
    random_state=42
)
twenty_test = fetch_20newsgroups(
    subset='test',
    categories=categories,
    shuffle=True,
    random_state=42
)

A basic text processing pipeline - bag of words features and Logistic Regression as a classifier:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, SGDClassifier
from sklearn.pipeline import make_pipeline

In [ ]:
vec = CountVectorizer()
clf = LogisticRegression(max_iter=1000)
pipe = make_pipeline(vec, clf)
pipe.fit(twenty_train.data, twenty_train.target);

Let's check quality of this pipeline:

In [ ]:
from sklearn import metrics

def print_report(pipe):
    y_test = twenty_test.target
    y_pred = pipe.predict(twenty_test.data)
    report = metrics.classification_report(y_test, y_pred, 
        target_names=twenty_test.target_names)
    print(report)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_test, y_pred)))
    
print_report(pipe)

                        precision    recall  f1-score   support

           alt.atheism       0.95      0.74      0.83       319
         comp.graphics       0.81      0.95      0.87       389
               sci.med       0.90      0.85      0.88       396
soc.religion.christian       0.88      0.93      0.90       398

              accuracy                           0.87      1502
             macro avg       0.88      0.87      0.87      1502
          weighted avg       0.88      0.87      0.87      1502

accuracy: 0.874


Not bad. We can try other classifiers and preprocessing methods, but let's check first what the model learned using ``eli5.show_weights`` function:

In [1]:
!pip install eli5

In [ ]:
import eli5

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
eli5.show_weights(clf, top=10)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


The table above doesn't make any sense; the problem is that eli5 was not able to get feature and class names from the classifier object alone. 

We would be able to solve this problem by providing **vectorizer** and let eli5 figure out the details automatically.

In [ ]:
eli5.show_weights(clf, vec=vec, top=10,
                  target_names=twenty_test.target_names)

This starts to make more sense. Columns are target classes. In each column there are features and their weights. 

Intercept (bias) feature is shown as ``<BIAS>`` in the same table. 

We can inspect features and weights because we're using a bag-of-words vectorizer and a linear classifier (so there is a direct mapping between individual words and classifier coefficients). 

Some features look good, but some don't. It seems model learned some names specific to a dataset (email parts, etc.) though, instead of learning topic-specific words. 

Let's check prediction results on an example:

In [ ]:
eli5.show_prediction(clf, twenty_test.data[0], vec=vec, 
                     target_names=twenty_test.target_names)

# probability is the value after softmax function
# score is score before softmax function.

What can be highlighted in text is highlighted in text. The separate table for features which can't be highlighted in text - ``<BIAS>`` in this case. If you hover mouse on a highlighted word it shows you a weight of this word in a title. Words are colored according to their weights.

## 2. Baseline model, improved data

From the highlighting above it can be seen that a classifier learned some non-interesting stuff indeed, e.g. it remembered parts of email addresses.

We should probably **clean the data first** to make it more interesting; improving model (trying different classifiers, etc.) doesn't make sense at this point - it may just learn to leverage these email addresses better. 

In practice we'd have to do cleaning yourselves; in this example 20 **newsgroups dataset provides an option to remove footers and headers** from the messages. Nice. Let's clean up the data and re-train a classifier.

In [ ]:
twenty_train = fetch_20newsgroups(
    subset='train',
    categories=categories,
    shuffle=True,
    random_state=42,
    remove=['headers', 'footers'],
)
twenty_test = fetch_20newsgroups(
    subset='test',
    categories=categories,
    shuffle=True,
    random_state=42,
    remove=['headers', 'footers'],
)

In [ ]:
vec = CountVectorizer()
clf = LogisticRegression(max_iter=1000)
pipe = make_pipeline(vec, clf)
pipe.fit(twenty_train.data, twenty_train.target);

We just made the task harder and more realistic for a classifier.

In [ ]:
print_report(pipe)

                        precision    recall  f1-score   support

           alt.atheism       0.80      0.76      0.78       319
         comp.graphics       0.81      0.93      0.86       389
               sci.med       0.86      0.77      0.81       396
soc.religion.christian       0.86      0.86      0.86       398

              accuracy                           0.83      1502
             macro avg       0.83      0.83      0.83      1502
          weighted avg       0.83      0.83      0.83      1502

accuracy: 0.832


A great result - we just made quality worse! Does it mean pipeline is worse now? No, likely it has a better quality on unseen messages. It is evaluation which is more fair now. Inspecting features used by classifier allowed us to notice a problem with the data and made a good change, despite of numbers which told us not to do that.

So, what have the updated classifier learned? (output is less verbose because only a subset of classes is shown - see "targets" argument):

In [ ]:
eli5.show_prediction(clf, twenty_test.data[0], vec=vec, 
                     target_names=twenty_test.target_names,
                     targets=['sci.med'])

It no longer uses email addresses (from: brian@ucsd.edu (brian kantor)), but it still doesn't look good: classifier assigns high weights to seemingly unrelated words like 'do' or 'my'. These words appear in many texts, so maybe classifier uses them as a proxy for bias. Or maybe some of them are more common in some of classes. 

## 3. Pipeline improvements

To help classifier we may filter out stop words:

In [ ]:
vec = CountVectorizer(stop_words='english')
clf = LogisticRegression(max_iter=1000)
pipe = make_pipeline(vec, clf)
pipe.fit(twenty_train.data, twenty_train.target)

print_report(pipe)

                        precision    recall  f1-score   support

           alt.atheism       0.85      0.77      0.81       319
         comp.graphics       0.84      0.94      0.89       389
               sci.med       0.92      0.85      0.88       396
soc.religion.christian       0.85      0.89      0.87       398

              accuracy                           0.87      1502
             macro avg       0.87      0.86      0.86      1502
          weighted avg       0.87      0.87      0.87      1502

accuracy: 0.866


In [ ]:
eli5.show_prediction(clf, twenty_test.data[0], vec=vec, 
                     target_names=twenty_test.target_names,
                     targets=['sci.med'])


Alternatively, we can use TF\*IDF scheme; it should give a somewhat similar effect. 



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer()
clf = LogisticRegression(max_iter=1000)
pipe = make_pipeline(vec, clf)
pipe.fit(twenty_train.data, twenty_train.target)

print_report(pipe)

                        precision    recall  f1-score   support

           alt.atheism       0.91      0.75      0.82       319
         comp.graphics       0.83      0.95      0.89       389
               sci.med       0.90      0.87      0.89       396
soc.religion.christian       0.85      0.88      0.87       398

              accuracy                           0.87      1502
             macro avg       0.87      0.86      0.87      1502
          weighted avg       0.87      0.87      0.87      1502

accuracy: 0.869


In [ ]:
eli5.show_prediction(clf, twenty_test.data[0], vec=vec, 
                     target_names=twenty_test.target_names,
                     targets=['sci.med'])

It helped, but didn't have quite the same effect. Why not do both?

In [ ]:
vec = TfidfVectorizer(stop_words='english')
clf = LogisticRegression(max_iter=1000)
pipe = make_pipeline(vec, clf)
pipe.fit(twenty_train.data, twenty_train.target)

print_report(pipe)

                        precision    recall  f1-score   support

           alt.atheism       0.93      0.74      0.82       319
         comp.graphics       0.90      0.97      0.93       389
               sci.med       0.92      0.92      0.92       396
soc.religion.christian       0.86      0.93      0.89       398

              accuracy                           0.90      1502
             macro avg       0.90      0.89      0.89      1502
          weighted avg       0.90      0.90      0.89      1502

accuracy: 0.896


In [ ]:
eli5.show_prediction(clf, twenty_test.data[0], vec=vec, 
                     target_names=twenty_test.target_names,
                     targets=['sci.med'])


## 4. Char-based pipeline

Let's try other analysers instead - use char n-grams instead of words:

In [ ]:
vec = TfidfVectorizer(stop_words='english', analyzer='char', 
                      ngram_range=(3,5))
clf = LogisticRegression(max_iter=500)
pipe = make_pipeline(vec, clf)
pipe.fit(twenty_train.data, twenty_train.target)

print_report(pipe)

                        precision    recall  f1-score   support

           alt.atheism       0.95      0.74      0.83       319
         comp.graphics       0.81      0.95      0.87       389
               sci.med       0.90      0.85      0.88       396
soc.religion.christian       0.88      0.93      0.90       398

              accuracy                           0.87      1502
             macro avg       0.88      0.87      0.87      1502
          weighted avg       0.88      0.87      0.87      1502

accuracy: 0.874


In [ ]:
eli5.show_prediction(clf, twenty_test.data[0], vec=vec, 
                     target_names=twenty_test.target_names)

It works, but quality is a bit worse. Also, it takes ages to train. 



If the number of words or features i.e. vocabulary is huge, consider using HashingVectorizer or perform Latent Semantic Analysis (Truncated SVD) and then apply classification algorithm.